# Genome-Wide linkage disequillibrium

Use this notebook to calculate different LD metrics across SNP data and plot ideogram

In [1]:
library(devtools)

Loading required package: usethis



In [2]:
#Install required packages
devtools::install_github('Rong-Zh/GWLD/GWLD-R')
devtools::install_github("SFUStatgen/LDheatmap")
devtools::install_github("sunnyeesl/BigLD")


RcppArmad... (0.12.8.3.0 -> 0.12.8.4.0) [CRAN]


Installing 1 packages: RcppArmadillo

Installing package into ‘/home/jupyter/packages’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpZc6Lm5/remotes1c032137280/Rong-Zh-GWLD-8ab4da7/GWLD-R/DESCRIPTION’ ... OK
* preparing ‘GWLD’:
* checking DESCRIPTION meta-information ... OK
* cleaning src
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘GWLD_1.3.6.tar.gz’



Installing package into ‘/home/jupyter/packages’
(as ‘lib’ is unspecified)

Skipping install of 'LDheatmap' from a github remote, the SHA1 (510c6005) has not changed since last install.
  Use `force = TRUE` to force installation

Skipping install of 'BigLD' from a github remote, the SHA1 (05d51086) has not changed since last install.
  Use `force = TRUE` to force installation



In [3]:
library(tidyverse)
library(LDheatmap)
library(GWLD)
library(BigLD)
library(colorspace) 
library(pdftools)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::%||%()   masks base::%||%()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘GWLD’


The following object is masked from ‘package:dplyr’:

    recode


Using poppler version 0.86.1



In [4]:
#LD blocking heatmap function
library(grid)


#' @export
LDblockHeatmap <- function(geno, SNPinfo, chrN, showSNPs = NULL, LDblockResult=NULL, tick = c("bp", "rsID"), st.bp=0 , ed.bp = Inf,
                          showLDsize = 3, savefile = FALSE, filename = "LDheatmap.tif"){
  # packagese
  # library(grid)
  ########################################################################################################
  # sub-Functions
  # 1. Big-LD    2. makeRect (built-in)    3. LDheatmap.Legend.add (built-in)
  ########################################################################################################
  # make rectangles
  makeRect<-function(nrow,ncol,cols,name,byrow=TRUE){
    xx<-(1:ncol)/ncol
    yy<-(1:nrow)/nrow
    right<-rep(xx,nrow)
    top<-rep(yy,each=ncol)
    rectGrob(x=right,y=top,width=1/ncol,height=1/nrow,just=c("right","top"),gp=gpar(col=NA,fill=cols),name=name)
  }
  # make Legend
  LDheatmap.Legend.add <- function(color, vp=VPheatmap){
    ImageRect <- makeRect(2, length(color), col = c(rep(NA, length(color)), color[length(color):1]), "colorKey")
    keyVP <- viewport(x = 0.5, y = 0.2, height = 0.4, width =0.5, just = c("centre", "bottom"), name = "keyVP")
    ttt <- expression(paste("r"^{2}, " Color Key"))
    title <- textGrob(ttt, x = 0.5, y = 1.25, name = "title", gp = gpar(cex = 0.6))
    labels <- textGrob(paste(0.2 * 0:5), x = 0.2 * 0:5, y = 0.25, gp = gpar(cex = 0.6), name = "labels")
    ticks <- segmentsGrob(x0 = c(0:5) * 0.2, y0 = rep(0.4,6), x1 = c(0:5) * 0.2, y1 = rep(0.5, 6), name = "ticks")
    box <- linesGrob(x = c(0, 0, 1, 1, 0), y = c(0.5, 1, 1, 0.5, 0.5), name = "box")
    key <- gTree(children = gList(ImageRect, title, labels, ticks, box), name = "Key", vp = keyVP)
    return(key)
  }
  ########################################################################################################
  if(length(tick) >1 ){
    tick = "rsID"
  }
  subSNPinfo = SNPinfo[which(SNPinfo[,2]>=st.bp & SNPinfo[,2]<=ed.bp),]
  if(dim(subSNPinfo)[1]>5000){
    print("There are Too many SNPs! We will draw only first 1000 SNPs")
    subSNPinfo<- subSNPinfo[1:1000,]
  }else if(dim(subSNPinfo)[1]<10){
    stop("Too short Region!")
  }
  if(!all(colnames(geno)==SNPinfo[,1])){
    stop("column names of geno data do not agree with 1st column of SNPinfo ")
  }

  chosencol = as.vector(match(as.character(subSNPinfo[,1]), colnames(geno)))
  subgeno = geno[,chosencol]
  if(is.null(LDblockResult)){
    subLDblockRes = Big_LD(subgeno, subSNPinfo)
    print("Big_LD, done!")
  }else{
    subLDblockRes = LDblockResult[which(LDblockResult$end.bp >= min(subSNPinfo[,2]) & LDblockResult$start.bp <= max(subSNPinfo[,2])),]
    subLDblockRes$start[1] = which(SNPinfo[,1]==subSNPinfo[1,1])
    levels(subLDblockRes$start.rsID) <- c(levels(subLDblockRes$start.rsID), as.character(subSNPinfo[1,1]))
    subLDblockRes$start.rsID[1] <- subSNPinfo[1,1]
    subLDblockRes$start.bp[1] = subSNPinfo[1,2]
    subsize = dim(subLDblockRes)[1]
    subSNPinfosize = dim(subSNPinfo)[1]
    subLDblockRes$end[subsize] = max(chosencol)
    levels(subLDblockRes$end.rsID) <- c(levels(subLDblockRes$end.rsID), as.character(subSNPinfo[subSNPinfosize,1]))
    subLDblockRes$end.rsID[subsize] <- as.character(subSNPinfo[subSNPinfosize,1])
    subLDblockRes$end.bp[subsize] = subSNPinfo[subSNPinfosize,2]
  }

  s = subLDblockRes$start
  e = subLDblockRes$end
  s.rsID = as.character(subLDblockRes$start.rsID)
  e.rsID = as.character(subLDblockRes$end.rsID)
  s.bp = subLDblockRes$start.bp
  e.bp = subLDblockRes$end.bp

  if(s.bp[1]<subSNPinfo[1,2]) {
    s.bp[1]<-subSNPinfo[1,2]
    s.rsID[1]<-as.character(subSNPinfo[1,1])
  }
  if(max(e.bp)>max(subSNPinfo[,2])){
    e.bp[length(e.bp)]<-max(subSNPinfo[,2])
    e.rsID[length(e.bp)]<-as.character(subSNPinfo[dim(subSNPinfo)[1],1])
  }

  subLDblockRes =  data.frame(s, e, s.rsID, e.rsID, s.bp, e.bp)
  colnames(subLDblockRes)=c("start", "end", "start.rsID", "end.rsID", "start.bp", "end.bp")

  BlockstP = sapply(subLDblockRes$start.bp, function(x) which(min(abs(x-subSNPinfo[,2])) ==abs(x-subSNPinfo[,2]))[1])-1
  BlockedP = sapply(subLDblockRes$end.bp, function(x) which(min(abs(x-subSNPinfo[,2])) ==abs(x-subSNPinfo[,2]))[1])
  if(!is.null(showSNPs)){
    showSNPsP = sapply(showSNPs[,2], function(x) which(abs(subSNPinfo[,2] - x) == min(abs(subSNPinfo[,2] - x)))[1])
  } else {
    showSNPsP = NULL
  }


  if(tick == "rsID"){
    tickname.st = as.character(subLDblockRes$start.rsID)
    tickname.ed = as.character(subLDblockRes$end.rsID)
  } else if(tick =="bp"){
    tickname.st = (subLDblockRes$start.bp)
    tickname.ed = (subLDblockRes$end.bp)
  }
  ticksizeNsatisfy = ((BlockedP-BlockstP+1)<showLDsize)
  print(ticksizeNsatisfy)
  BlockstP.tick = BlockstP[!ticksizeNsatisfy]
  BlockedP.tick = BlockedP[!ticksizeNsatisfy]
  tickname.st = tickname.st[!ticksizeNsatisfy]
  tickname.ed = tickname.ed[!ticksizeNsatisfy]
  VPheatmap<-viewport(x=0.05,y=0.1,width=unit(0.85,"snpc"),height=unit(0.85,"snpc"),just=c("left","bottom"), name="VPheatmap")
  VPtick<-viewport(x=0.055,y=0.095,width=unit(0.85,"snpc"),height=unit(0.85,"snpc"),just=c("left","bottom"), name="VPtick")
  VPLegend<-viewport(x=0.65,y=0,width=unit(0.35,"snpc"),height=unit(0.15,"snpc"),just=c("left","bottom"), name="VPLegend")
  ###make rectangle
  color=heat.colors(50)
  M = cor(subgeno)^2
  M[lower.tri(M,diag=FALSE)]<-NA
  mybreak<-0:length(color)/length(color)
  colcut<-as.character(cut(1-M,mybreak,labels=as.character(color), include.lowest=TRUE, include.highest=FALSE))
  ldheatmap<-makeRect(dim(M)[1],dim(M)[2],colcut,"ldheatmap")
  WhiteTri = polygonGrob(x=c(0, 1, 1), y=c(0, 0, 1),
                         id=NULL, id.lengths=NULL, name=NULL, gp=gpar(col = "white", fill = "white"))
  x1s = BlockstP*(1/dim(M)[1])
  x2s = BlockstP*(1/dim(M)[1])
  x3s = BlockedP*(1/dim(M)[1])
  y1s = BlockstP*(1/dim(M)[1])
  y2s = BlockedP*(1/dim(M)[1])
  y3s = BlockedP*(1/dim(M)[1])
  trascolor =   adjustcolor( "red", alpha.f = 0)
  BlockBoundaries = polygonGrob(x = c(x1s, x2s, x3s), y = c(y1s, y2s, y3s), id = rep(1:length(x1s),3), gp = gpar(fill = trascolor))
  blockname1 = tickname.st
  blockname2 = tickname.ed
  tickposi1 =  BlockstP.tick*(1/dim(M)[1])+0.005
  tickposi2 = BlockedP.tick*(1/dim(M)[1])-0.005
  tickname = c(tickname.st, tickname.ed)
  tickposi = c(tickposi1, tickposi2)
  print(length(tickname))
  LDticks = textGrob(tickname, x = tickposi, y = tickposi, just = c("left", "centre"),rot = -45, gp = gpar(cex = 0.5), vp = VPtick)
  LDblockHeatmap = gTree(children=gList(ldheatmap, WhiteTri, BlockBoundaries),name="LDHeatmap", vp = VPheatmap)
  if(is.null(showSNPsP)){
    showSNPposi = NULL
    LDblockMap = gTree(children=gList(LDblockHeatmap, LDticks),name="LDHeatmap")
  }else{
    showSNPposi = showSNPsP*(1/dim(M)[1])
    SNPticks = textGrob(as.character(showSNPs[,1]), x = showSNPposi+0.075, y = showSNPposi-0.075,
                        just = c("left", "centre"), gp = gpar(cex = 0.5, col = "red"), vp = VPtick)
    SNPseg = segmentsGrob(x0 =showSNPposi , y0=showSNPposi, x1= showSNPposi+0.07, y1=showSNPposi-0.07, gp = gpar(col = "red", alpha = 0.5),vp = VPtick )
    LDblockMap = gTree(children=gList(LDblockHeatmap,  SNPseg, SNPticks ,LDticks),name="LDHeatmap")
  }

  Legend = gTree(children = gList(LDheatmap.Legend.add(color=heat.colors(50))), vp = VPLegend)

  regionname = paste("chr",chrN,": ",format(min(subSNPinfo[,2]), big.mark=",",scientific=FALSE) ,
                     "bp ~ ", format(max(subSNPinfo[,2]), big.mark=",",scientific=FALSE),"bp", sep="" )

  Figregion = textGrob(regionname, x = 0.05, y = 0.98, just = c("left", "centre"),gp = gpar(cex = 1))
  TotalMap = gTree(children = gList(LDblockMap, Figregion, Legend))
  # grid.draw(LDblockMap)

  if(savefile ==FALSE){
    grid.draw(TotalMap)
    return(TotalMap)
  } else {
    fileN = filename
    pdf(file=fileN, width=11, height=8)
    grid.draw(TotalMap)
    dev.off()
    # grid.draw(TotalMap)
    # return(TotalMap)
  }
}

In [5]:
#read in genotype data for MHC
SNP <- GWLD::read.plink("LD_plink")


#prepare snp info
Info <- SNP$bim
Info <- Info[, c(1, 4, 2)]
colnames(Info) <- c("CHROM", "POS", "ID")
geno012 <- SNP$bed

num_cores <- 16

In [19]:
#calculate different LD metrics
r2 <- LD(geno012, cores = num_cores)
mi <- MI(geno012, cores = num_cores)
RMI <- RMI(geno012, cores = num_cores)


In [ ]:
#write metrics to file
write.csv(r2,'linkage_r2.csv')

write.csv(mi,'linkage_mi.csv')
write.csv(RMI,'linkage_RMI.csv')

In [ ]:
#filter plink fileset for MHC region for plotting
system("plink --bfile LD_plink --from chr6:26090042:G:C --to chr6:33105545:A:G --make-bed --out MHC_LD",intern=T)

In [6]:
library(snpStats)

#read plink file for ld heatmap drawing
SNP <- snpStats::read.plink("MHC_LD")

Loading required package: survival

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack



Attaching package: ‘snpStats’


The following object is masked from ‘package:GWLD’:

    read.plink




In [7]:
#prepare genotypes
geno <- as.data.frame(SNP$genotypes)
geno[] <- lapply(geno, as.numeric)
SNPinfo <- SNP$map %>% select(c(snp.name,position))
geno <- 3-geno
geno

,chr6:26090042:G:C,chr6:26090042:G:T,chr6:26326086:G:A,chr6:26326086:G:C,chr6:26328125:T:C,chr6:26422399:C:A,chr6:26970051:C:A,chr6:27373692:A:C,chr6:27375506:C:A,chr6:27377052:C:G,⋯,chr6:33088986:A:G,chr6:33088988:C:T,chr6:33089139:G:T,chr6:33089399:T:C,chr6:33089421:G:C,chr6:33089663:C:T,chr6:33089934:G:A,chr6:33090058:C:T,chr6:33095626:A:G,chr6:33105545:A:G
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1000131,0,0,1,0,0,0,0,2,1,1,⋯,0,0,0,0,0,0,0,0,0,0
1000479,0,0,1,0,0,0,0,1,1,1,⋯,1,1,1,1,1,1,1,1,1,1
1000803,1,0,0,0,0,0,0,1,1,1,⋯,2,2,2,2,2,2,2,2,2,2
1000982,0,0,0,0,0,0,0,1,0,0,⋯,0,0,0,0,0,0,0,0,0,0
1001136,0,0,1,0,0,0,0,2,1,1,⋯,0,0,0,0,0,0,0,0,0,0
1001259,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
1001315,0,0,0,1,1,1,1,0,0,0,⋯,1,1,1,1,1,1,1,1,1,1
1001705,0,0,0,0,0,0,0,2,2,2,⋯,1,1,1,1,1,1,1,1,1,1
1001837,0,0,0,0,0,0,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [10]:
#Use BigLD to calculate LD blocks, with cuttoff r^2 of 0.8
res1 <- Big_LD(geno, SNPinfo,  CLQcut = 0.8)

res1 %>% write_csv('Big_LDranges.csv')

start,end,start.rsID,end.rsID,start.bp,end.bp,blocksize
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
549,803,chr6:31329719:C:T,chr6:31350655:T:C,31329719,31350655,254
1204,1339,chr6:32412431:A:G,chr6:32461085:T:C,32412431,32461085,135
1420,1966,chr6:32590240:A:C,chr6:32615228:C:G,32590240,32615228,546
1993,3117,chr6:32619336:G:A,chr6:32711549:C:T,32619336,32711549,1124
3246,3452,chr6:33079254:T:C,chr6:33105545:A:G,33079254,33105545,206


In [8]:
res1 <- read_csv('Big_LDranges.csv')

Rows: 67 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): start.rsID, end.rsID
dbl (4): start, end, start.bp, end.bp

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [9]:
#Draw heatmap with LD blocks plotted to visualize useful LD blocks
LDblockHeatmap(geno = geno, SNPinfo = SNPinfo,chrN=6,tick='bp',st.bp=26087429,LDblockResult=res1,ed.bp=33589696,showLDsize = 100,savefile = TRUE,filename='LD_block_Heatmap_MHC.pdf')

[1] FALSE
[1] 2


png 
  2

In [9]:
#Draw heatmap with LD blocks plotted to visualize useful LD blocks
LDblockHeatmap(geno = geno, SNPinfo = SNPinfo,chrN=6,tick='bp',st.bp=33079254,LDblockResult=res1,ed.bp=33105545,showLDsize = 100,savefile = TRUE,filename='LD_block_Heatmap_33079254_33105545_MHC.pdf')

[1] FALSE
[1] 2


png 
  2

In [9]:
#prepare r^2 results for linkage plot
r2 <- read_csv('linkage_r2.csv')

New names:
• `` -> `...1`
Rows: 3580 Columns: 3581
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr    (1): ...1
dbl (3579): chr1:47155607:T:A, chr1:221607617:T:C, chr1:236151286:C:T, chr2:...
lgl    (1): chr1:47150227:G:T

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [10]:
#filter linkage results for MHC region
rows<-r2$'...1'
r2 <- subset(r2, select = -1)
rownames(r2)<- rows
r2

MHC_r3<-r2[startsWith(colnames(r2),'chr6'),startsWith(rownames(r2),'chr6')][-1,-1][-3453,-3453]

rownames(MHC_r3)<-rows[grepl('chr6',rows)][-1][-3453]

Warning message:
“Setting row names on a tibble is deprecated.”


,chr1:47150227:G:T,chr1:47155607:T:A,chr1:221607617:T:C,chr1:236151286:C:T,chr2:3765493:T:C,chr2:3765517:T:G,chr2:3766716:A:G,chr2:10864363:G:A,chr2:106348182:C:T,chr2:106348617:C:T,⋯,chr19:44405935:C:A,chr19:52582779:CAA:TAA,chr19:52584239:A:G,chr20:869010:G:A,chr20:54038109:T:C,chr21:42022276:C:T,chr21:42022619:A:G,chr21:42022620:C:A,chr21:42022658:G:A,chr21:42023004:A:C
,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1:47150227:G:T,NA,0.9959269,0.0002708070,4.476697e-04,5.409164e-04,5.497506e-04,4.223116e-05,3.248684e-05,9.036490e-04,9.135745e-04,⋯,4.565104e-05,1.082060e-03,1.075886e-03,4.076366e-04,1.074225e-05,1.955561e-04,2.075630e-04,2.067534e-04,1.593883e-04,3.791770e-04
chr1:47155607:T:A,NA,NA,0.0002605228,4.392493e-04,4.369466e-04,4.449732e-04,2.823649e-05,3.313058e-05,8.460632e-04,8.556063e-04,⋯,3.870350e-05,9.140482e-04,9.292458e-04,4.574292e-04,5.200371e-06,1.802038e-04,2.012606e-04,2.004592e-04,1.696765e-04,3.613232e-04
chr1:221607617:T:C,NA,NA,NA,7.837590e-07,4.526784e-06,1.665097e-06,2.571241e-08,2.831538e-05,9.292053e-05,9.435888e-05,⋯,4.119307e-05,6.979867e-07,7.741321e-07,7.939276e-05,2.550367e-06,9.258183e-11,1.060940e-08,1.171646e-08,1.986330e-07,1.074809e-07
chr1:236151286:C:T,NA,NA,NA,NA,1.278465e-04,1.299384e-04,1.879985e-05,6.204055e-04,1.172197e-04,1.138889e-04,⋯,1.297164e-04,5.289633e-04,5.258618e-04,1.269107e-04,2.868706e-07,4.808870e-05,3.472603e-05,3.458632e-05,3.753131e-05,8.832928e-05
chr2:3765493:T:C,NA,NA,NA,NA,NA,9.987871e-01,9.130869e-01,7.639086e-05,2.973023e-05,3.654577e-05,⋯,2.639995e-04,1.742643e-03,1.725729e-03,9.343994e-05,1.255484e-04,1.351430e-04,1.411991e-04,1.418123e-04,2.913377e-04,1.065385e-04
chr2:3765517:T:G,NA,NA,NA,NA,NA,NA,9.126832e-01,7.580608e-05,3.035919e-05,3.724474e-05,⋯,2.740033e-04,1.761625e-03,1.744657e-03,9.482812e-05,1.269413e-04,1.379790e-04,1.440968e-04,1.447156e-04,2.954324e-04,1.090887e-04
chr2:3766716:A:G,NA,NA,NA,NA,NA,NA,NA,3.313182e-06,8.427398e-05,9.115830e-05,⋯,2.016738e-05,2.443879e-04,2.441436e-04,2.666399e-05,6.436251e-05,8.235672e-04,7.811234e-04,7.825996e-04,5.756379e-04,9.692426e-04
chr2:10864363:G:A,NA,NA,NA,NA,NA,NA,NA,NA,1.142609e-04,1.081177e-04,⋯,3.094028e-04,9.562796e-06,9.228999e-06,4.654338e-06,2.951299e-05,5.697918e-05,5.529304e-05,5.564639e-05,8.266559e-05,5.077685e-05
chr2:106348182:C:T,NA,NA,NA,NA,NA,NA,NA,NA,NA,9.980365e-01,⋯,8.722105e-06,2.970615e-04,2.910004e-04,2.631337e-04,5.580790e-05,2.642981e-06,6.691417e-06,6.555625e-06,5.239576e-06,1.302503e-05


Warning message:
“Setting row names on a tibble is deprecated.”


In [11]:
mhc_map <- (read.table('LD_plink.bim')%>% filter(V1==6 & V4>26000000 & V4<34000000))$V4

In [41]:
save(ld_blocks, file = "ld_blocks.RData")


In [12]:
load("ld_blocks.RData")